# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 11:39AM,264658,19,9568677,"Emersa Waterbox, LLC",Released
1,Jun 13 2023 11:34AM,264654,19,ADV80021042,"AdvaGen Pharma, Ltd",Released
2,Jun 13 2023 11:34AM,264654,19,ADV80021043,"AdvaGen Pharma, Ltd",Released
3,Jun 13 2023 11:34AM,264654,19,ADV80020984,"AdvaGen Pharma, Ltd",Released
4,Jun 13 2023 11:34AM,264654,19,ADV80020985,"AdvaGen Pharma, Ltd",Released
5,Jun 13 2023 11:34AM,264654,19,ADV80020986,"AdvaGen Pharma, Ltd",Released
6,Jun 13 2023 11:34AM,264654,19,ADV80020987,"AdvaGen Pharma, Ltd",Released
7,Jun 13 2023 11:34AM,264654,19,ADV80020988,"AdvaGen Pharma, Ltd",Released
8,Jun 13 2023 11:34AM,264654,19,ADV80021033,"AdvaGen Pharma, Ltd",Released
9,Jun 13 2023 11:34AM,264654,19,ADV80020978,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,264654,Released,26
19,264655,Released,14
20,264656,Released,1
21,264657,Released,1
22,264658,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,NaN,NaN,26.0
264655,NaN,NaN,14.0
264656,NaN,NaN,1.0
264657,NaN,NaN,1.0
264658,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264559,0.0,22.0,6.0
264604,0.0,1.0,0.0
264620,5.0,10.0,3.0
264622,0.0,1.0,0.0
264627,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264559,0,22,6
264604,0,1,0
264620,5,10,3
264622,0,1,0
264627,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264559,0,22,6
1,264604,0,1,0
2,264620,5,10,3
3,264622,0,1,0
4,264627,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264559,,22,6
1,264604,,1,
2,264620,5,10,3
3,264622,,1,
4,264627,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC"
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd"
27,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC"
28,Jun 13 2023 11:31AM,264656,10,Emerginnova
29,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC
43,Jun 13 2023 11:23AM,264653,10,"Methapharm, Inc."
52,Jun 13 2023 11:21AM,264652,10,"Methapharm, Inc."
53,Jun 13 2023 11:19AM,264651,10,MedStone Pharma LLC
56,Jun 13 2023 11:16AM,264649,19,"Exact-Rx, Inc."
57,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC",,,1
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",,,26
2,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",,,1
3,Jun 13 2023 11:31AM,264656,10,Emerginnova,,,1
4,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,,,14
5,Jun 13 2023 11:23AM,264653,10,"Methapharm, Inc.",,,9
6,Jun 13 2023 11:21AM,264652,10,"Methapharm, Inc.",,,1
7,Jun 13 2023 11:19AM,264651,10,MedStone Pharma LLC,,,3
8,Jun 13 2023 11:16AM,264649,19,"Exact-Rx, Inc.",,,1
9,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,,3,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC",1,,
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",26,,
2,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 13 2023 11:31AM,264656,10,Emerginnova,1,,
4,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,14,,
5,Jun 13 2023 11:23AM,264653,10,"Methapharm, Inc.",9,,
6,Jun 13 2023 11:21AM,264652,10,"Methapharm, Inc.",1,,
7,Jun 13 2023 11:19AM,264651,10,MedStone Pharma LLC,3,,
8,Jun 13 2023 11:16AM,264649,19,"Exact-Rx, Inc.",1,,
9,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,3,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC",1,,
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",26,,
2,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 13 2023 11:31AM,264656,10,Emerginnova,1,,
4,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",26.0,NaN,NaN
2,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Jun 13 2023 11:31AM,264656,10,Emerginnova,1.0,NaN,NaN
4,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 11:39AM,264658,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",26.0,0.0,0.0
2,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Jun 13 2023 11:31AM,264656,10,Emerginnova,1.0,0.0,0.0
4,Jun 13 2023 11:27AM,264655,10,MedStone Pharma LLC,14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2117198,43.0,3.0,0.0
15,264559,6.0,22.0,0.0
16,1058509,3.0,1.0,0.0
19,1058581,31.0,10.0,5.0
21,264644,0.0,1.0,0.0
22,264622,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2117198,43.0,3.0,0.0
1,15,264559,6.0,22.0,0.0
2,16,1058509,3.0,1.0,0.0
3,19,1058581,31.0,10.0,5.0
4,21,264644,0.0,1.0,0.0
5,22,264622,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,43.0,3.0,0.0
1,15,6.0,22.0,0.0
2,16,3.0,1.0,0.0
3,19,31.0,10.0,5.0
4,21,0.0,1.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,43.0
1,15,Released,6.0
2,16,Released,3.0
3,19,Released,31.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,0.0,0.0,0.0,5.0,0.0,0.0
Executing,3.0,22.0,1.0,10.0,1.0,1.0
Released,43.0,6.0,3.0,31.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,3.0,22.0,1.0,10.0,1.0,1.0
2,Released,43.0,6.0,3.0,31.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,3.0,22.0,1.0,10.0,1.0,1.0
2,Released,43.0,6.0,3.0,31.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()